<h1><center>Case Study - B3</center></h1>

# Overview & Cleaning

In [1]:
import pandas as pd

In [2]:
# loading the df
df = pd.read_csv("base_case.csv", sep=';');

|Nome          |Período|Unidade|Descrição                        |
|:-------------|-----  |------|--------------------------------- |
|id_cliente    |N/A    |N/A   |Unique id that identifies customer  |
|renda         |N/A    |R\$   |Declared income range          |
|profissao     |N/A    |N/A   |Group of professions grouped     |
|val_compra    |M0     |R\$    |Total value of purchased shares   |
|val_venda     |M0     |R\$    |Total value of shares sold    |
|tm_compra     |M0     |R\$   |Value per share - purchase ("ticket")|
|tm_venda      |M0     |R\$    |Value per share - sale ("ticket") |
|acoes         |M0     |#     |# shares in the portfolio               |
|acoes_dif     |M0     |#     |# different shares in the portfolio    |
|ativo_m1      |M1     |Flag  |1=Customer is active in M1        |
|ligou_cr_m1   |M1     |Flag  |1=Contact from sales team in M1        |

In [3]:
df.head() # viewing the header and first lines

,id_cliente,renda,profissao,val_compra,val_venda,tm_compra,tm_venda,acoes,acoes_dif,ativo_m1,ligou_cr_m1
0,280249781,De 0 a 5k,B,0,"2941,2573",NaN,"29,412573",9900,40,1,0
1,706585147,De 5 a 10k,A,0,"9388,480268",NaN,"15,64746711",7800,20,1,0
2,968252151,De 0 a 5k,A,0,0,NaN,NaN,1600,5,0,1
3,395769633,De 5 a 10k,C,0,"4488,875068",NaN,"11,22218767",7200,30,1,0
4,909158440,De 5 a 10k,B,"3340,069458","4143,588932","16,70034729","20,71794466",4400,10,1,1


In [4]:
# to understand general dataframe information
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_cliente   10000 non-null  int64 
 1   renda        10000 non-null  object
 2   profissao    10000 non-null  object
 3   val_compra   10000 non-null  object
 4   val_venda    10000 non-null  object
 5   tm_compra    6006 non-null   object
 6   tm_venda     5966 non-null   object
 7   acoes        10000 non-null  int64 
 8   acoes_dif    10000 non-null  int64 
 9   ativo_m1     10000 non-null  int64 
 10  ligou_cr_m1  10000 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 859.5+ KB


(10000, 11)

There are null values in the **tm_compra** and **tm_venda** columns

In [5]:
df.isnull().sum() # confirming how many null values

id_cliente        0
renda             0
profissao         0
val_compra        0
val_venda         0
tm_compra      3994
tm_venda       4034
acoes             0
acoes_dif         0
ativo_m1          0
ligou_cr_m1       0
dtype: int64

In [6]:
# understanding what (unique) values we have in columns that appear to be categorical
# K means reading elements in the order they occur in memory.
# ravel returns an adjacent flat 1-D array containing the elements

pd.unique(df[['renda', 'profissao', 'ativo_m1', 'ligou_cr_m1']].values.ravel('K'))

array(['De 0 a 5k', 'De 5 a 10k', 'De 15 a 20k', 'De 10 a 15k', 'B', 'A',
       'C', 1, 0], dtype=object)

In [7]:
# return sum of duplicate rows
sum(df.duplicated(keep='first'))

0

### Notes for treatment
- There are missing values only in *tm_compra* and *tm_venda* (expected, as there was not necessarily a purchase or sale in M0)
- No duplicate lines
- Convert datatype from *id_cliente* to object
- Convert datatype from *renda*, *profissao* to categorical, if applying ML
- Convert the datatype of the value and average ticket columns to float
- Convert datatype of flags to bool


In [8]:
# convert datatype from 'id_cliente' to object

df['id_cliente']=df['id_cliente'].astype(str)

In [9]:
# convert datatype from 'renda', 'profissao' to categorical if applying ML

to_convert1 = ['renda', 'profissao']
df[to_convert1] = df[to_convert1].astype('category')

In [10]:
# convert the datatype of the value and average ticket columns to float

# BEFORE - replace null values in 'tm_compra' and 'tm_venda' to 0
# to be able to convert all ',' in these columns to '.' - and turn them into numerical variables
df.update(df[['tm_compra','tm_venda']].fillna(0))

import re #Regular expression operations - allows special characters to be used without having their senses invoked

to_convert2 = ['val_compra', 'val_venda', 'tm_compra', 'tm_venda']

for col in to_convert2:
    df[col] = pd.to_numeric(df[col].apply(lambda x: re.sub(',', '.', str(x)))).round(2)

In [11]:
# convert flags datatype to bool

to_convert3 = ['ativo_m1', 'ligou_cr_m1']
df[to_convert3] = df[to_convert3].astype('bool')

Result:

In [12]:
df.head()

,id_cliente,renda,profissao,val_compra,val_venda,tm_compra,tm_venda,acoes,acoes_dif,ativo_m1,ligou_cr_m1
0,280249781,De 0 a 5k,B,0.00,2941.26,0.0,29.41,9900,40,True,False
1,706585147,De 5 a 10k,A,0.00,9388.48,0.0,15.65,7800,20,True,False
2,968252151,De 0 a 5k,A,0.00,0.00,0.0,0.00,1600,5,False,True
3,395769633,De 5 a 10k,C,0.00,4488.88,0.0,11.22,7200,30,True,False
4,909158440,De 5 a 10k,B,3340.07,4143.59,16.7,20.72,4400,10,True,True


In [13]:
df.info() # note that there are no more null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   id_cliente   10000 non-null  object  
 1   renda        10000 non-null  category
 2   profissao    10000 non-null  category
 3   val_compra   10000 non-null  float64 
 4   val_venda    10000 non-null  float64 
 5   tm_compra    10000 non-null  float64 
 6   tm_venda     10000 non-null  float64 
 7   acoes        10000 non-null  int64   
 8   acoes_dif    10000 non-null  int64   
 9   ativo_m1     10000 non-null  bool    
 10  ligou_cr_m1  10000 non-null  bool    
dtypes: bool(2), category(2), float64(4), int64(2), object(1)
memory usage: 586.4+ KB


### New features

Three new columns that we can take intelligence from: number of shares bought, number of shares sold, and the difference between the two - by calculating:

\begin{equation*}
qt\_compras =
\frac{val\_compra}{tm\_compra}
\quad\quad 
qt\_vendas =
\frac{val\_venda}{tm\_venda}
\quad\quad 
\end{equation*}
<br>
\begin{align}
\mathbf{dif\_cv} & = \mathbf{qt\_compras}-\mathbf{qt\_vendas}
\end{align}
<br>

In [14]:
# applying forums and creating columns
df['qt_compras'] = (df['val_compra'] / df['tm_compra']).round(2)
df['qt_vendas'] = (df['val_venda'] / df['tm_venda']).round(2)

# Replace null values in 'qt_compras' and 'qt_vendas' to 0
df.update(df[['qt_compras','qt_vendas']].fillna(0))

df['dif_cv'] = (df['qt_compras'] - df['qt_vendas'])

In [15]:
# organizing df columns
df = df[['id_cliente','renda', 'profissao', 'qt_compras', 'val_compra','tm_compra','qt_vendas',
               'val_venda','tm_venda','dif_cv','acoes','acoes_dif','ativo_m1','ligou_cr_m1']]
df.head(2)

,id_cliente,renda,profissao,qt_compras,val_compra,tm_compra,qt_vendas,val_venda,tm_venda,dif_cv,acoes,acoes_dif,ativo_m1,ligou_cr_m1
0,280249781,De 0 a 5k,B,0.0,0.0,0.0,100.01,2941.26,29.41,-100.01,9900,40,True,False
1,706585147,De 5 a 10k,A,0.0,0.0,0.0,599.90,9388.48,15.65,-599.90,7800,20,True,False


**Saving the file for exploratory analysis**

In [16]:
df.to_csv("df_cleaned.csv", date_format='%Y-%m-%d', index=False)